In [6]:
# %load bookQA.py
import re
import pandas as pd
from openai import OpenAI
import time

start_time = time.time()
class OpenAILLM():
    def __init__(self, api_key, base_url, kwargs):                  # __init__ is the constructor of class OpenAILLM() for initialization
        self.client = OpenAI(api_key=api_key, base_url=base_url)    # created an instance of thiis class and stored it in the self. client attribute
        self.kwargs = kwargs                                        # the additional param that may store API calls. it refers the temp param according to below.

    def generate(self, prompt, **kwargs):                                            # generate method is designed to create text
        messages = [{"role": "user", "content": prompt}]                             
        # create a list which includes single dictionary. It contains two keys which accept info from users and prompt respectively. 
        
        response = self.client.chat.completions.create(model="gpt-3.5-turbo-1106",
                                                       messages=messages,
                                                       **kwargs)
        # The call to this method will send a request to OpenAI and return the generated text.
        
        result = response.choices[0].message.content     # extract generated text from response
        return result


def get_prompt(subject, file_path):
    with open(file_path + str(subject) + "fewshot.txt", 'r', encoding='utf-8') as fewshot_file, \
            open(file_path + "prompt.txt", 'r', encoding='utf-8') as prompt_file:
                # Build a complete file path that points to the prescriptive file.
                
        fewshot = fewshot_file.read()
        prompt = prompt_file.read()

    prompt = prompt.replace('{subject}', subject)
    prompt = prompt.replace('{fewshot}', fewshot)
    return prompt


def get_response(api_key, base_url, prompt, **kwargs):
    gpt = OpenAILLM(api_key, base_url, kwargs)
    response = gpt.generate(prompt)
    return response                   # This method sends a request to the OpenAI API and obtains the returned response.


def main():
    api_key = "YOUR KEY"
    base_url = "YOUR URL"
    kwargs = {
        "temperature": 0.3
    }
    
    # 运行之前需要核对
    subject = "金融学"
    file_path = "/Users/anbosmac/Documents/present/intern_n_project/Knowledge_Consulting/API2GPT/3.14-trial/"
    
    # 生成初始prompt
    prompt_template = get_prompt(subject, file_path)

    # 读取context
    df = pd.read_csv(file_path + str(subject) + "context-1.5-0.5(k).csv", encoding='utf-8')
    
    # 准备存放结果的DataFrame
    if 'amount' not in df.columns:
        df['amount'] = 0  # 如果不存在amount列，则添加
    
    for index, row in df.iterrows():
        context = row['context']
        prompt = prompt_template.replace('{context}', context)
        
        # 调用API生成语料
        response = get_response(api_key, base_url, prompt, **kwargs)
        
        # 提取题目和答案
        pattern = r"题目\d+：(.*?)\n答案\d+：(.*?)\n"
        matches = re.findall(pattern, response)
        
        # 更新DataFrame
        df.at[index, 'amount'] = len(matches)
        
        # 准备保存的数据
        data = {'题目': [match[0].strip() for match in matches], '答案': [match[1].strip() for match in matches]}
        output = pd.DataFrame(data)
        
        # 追加到output.csv
        output.to_csv(file_path + 'output-1.5-0.5(k).csv', mode='a', header=False, index=False, encoding='utf-8-sig')
    
    # 更新context.csv
    df.to_csv(file_path + str(subject) + "context-1.5-0.5(k).csv", index=False, encoding='utf-8-sig')

if __name__ == "__main__":
    main()

end_time = time.time()
elapsed_time = end_time - start_time
hours = elapsed_time // 3600
minutes = (elapsed_time % 3600) // 60
seconds = elapsed_time % 60
print(f"Total runtime: {int(hours)} hours {int(minutes)} mins {int(seconds)} seconds")

Total runtime: 0 hours 3 mins 41 seconds
